In [1]:
words = open('../names.txt','r').read().splitlines()

In [2]:
import torch

In [28]:
# stoi and itos dicts
chars = ['.']+sorted([chr(char) for char in range(ord('a'),ord('z')+1)])+['<s>']
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

In [19]:
N = torch.zeros(28,28,27) # cuz I wouldn't expect <s> for the 3rd dimension. and anyway its just a filler character.

In [20]:
#filling the N array.
for word in words:
    chs = ['.']+['<s>']+list(word)+['.']
    for cha,chb,chc in zip(chs,chs[1:],chs[2:]):
        # print(f'cha,chb,chc : {cha,chb,chc}')
        i,j,k = stoi[cha],stoi[chb],stoi[chc]
        N[i,j,k]+=1

In [21]:
N.sum() # the count will be same as you can imagine each increment is like sliding over the name and turns out with your current approach you are sliding as much as when you were doing bigrams with just window size of 3.

tensor(228146.)

In [ ]:
P = (N+1).float()
# print(P.sum(dim=2).shape) #wrong # torch.Size([28, 28])
# print(P.shape) # torch.Size([28, 28, 27])
# P.sum(dim=2,keepdim=True).shape #torch.Size([28, 28, 1])
P/=P.sum(dim=2,keepdim=True)

In [24]:
log_likelihood=0.0
ncount=0
for w in words:
    chs = ['.']+['<s>']+list(word)+['.']
    for cha,chb,chc in zip(chs,chs[1:],chs[2:]):
        i,j,k = stoi[cha],stoi[chb],stoi[chc]
        prob = P[i,j,k]
        log_likelihood+=torch.log(prob)
        ncount+=1
neg_log_likelihood = -log_likelihood
mean_neg_log_likelihood = neg_log_likelihood/ncount

In [25]:
print(f'{neg_log_likelihood=}')
print(f'{mean_neg_log_likelihood=}')

neg_log_likelihood=tensor(735231.7500)
mean_neg_log_likelihood=tensor(3.8254)


In [34]:
g = torch.Generator().manual_seed(2147483647+27)


for _ in range(10):
    nameList = []
    ix,iy =0,27
    while True:
        p = P[ix,iy]
        iz = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        nameList.append(itos[iz])
        ix,iy=iy,iz
    print(''.join(nameList))

brianie
timara
royene
abhjpper
katten
ne
gehilam
maylyn
ger
ney


Implementing the neural net.

In [35]:
nnNames = list(words)

In [36]:
import random
random.shuffle(nnNames)

train_size = int(0.8 * len(nnNames))
dev_size = int(0.1 * len(nnNames))
test_size = len(nnNames) - train_size - dev_size

train_data = nnNames[:train_size]
dev_data = nnNames[train_size:train_size + dev_size]
test_data = nnNames[train_size + dev_size:]

In [37]:
xs,ys,zs = [],[],[]

for name in nnNames:
    n = ['.']+['<s>']+list(word)+['.']
    for cha,chb,chc in zip(n,n[1:],n[2:]):
        xs.append(stoi[cha])
        ys.append(stoi[chb])
        zs.append(stoi[chc])

xs,ys,zs = torch.tensor(xs),torch.tensor(ys),torch.tensor(zs)

In [38]:
xs.shape,ys.shape,zs.shape

(torch.Size([192198]), torch.Size([192198]), torch.Size([192198]))

In [39]:
import torch.nn.functional as F

In [40]:
# was thinking like how we used W as the same dimension as N in bigram I think I should do the same here
W = torch.randn((28,28,27),requires_grad=True)


In [51]:
cpyW = W.clone().detach()

Backpropagation

In [41]:
zenc = F.one_hot(zs,num_classes=27).float()
zenc.shape

torch.Size([192198, 27])

In [46]:
W[1,1,:]

tensor([-2.8728,  0.2079,  0.3033,  0.4602, -0.4113, -0.5321, -0.3622, -0.2333,
        -1.4056, -1.9979, -1.1313,  0.5306,  1.0756, -0.1519,  2.6782,  1.3059,
        -1.2850, -0.6498, -2.7589,  0.5130, -1.5979,  0.4884, -0.1390,  0.6705,
        -0.5244, -1.2992, -0.1628], grad_fn=<SliceBackward0>)

In [47]:
W[[1,2],[1,2]]

tensor([[-2.8728,  0.2079,  0.3033,  0.4602, -0.4113, -0.5321, -0.3622, -0.2333,
         -1.4056, -1.9979, -1.1313,  0.5306,  1.0756, -0.1519,  2.6782,  1.3059,
         -1.2850, -0.6498, -2.7589,  0.5130, -1.5979,  0.4884, -0.1390,  0.6705,
         -0.5244, -1.2992, -0.1628],
        [-1.0330, -2.0707,  0.5708,  0.2341,  0.0701,  0.3084,  0.5823,  2.1389,
          1.2318, -0.0861,  0.8318,  0.0696,  0.9047,  0.5470, -0.6897, -0.2083,
         -0.5043, -0.6823, -0.7821, -0.6801, -2.0858, -0.0210,  0.4577, -0.5234,
         -0.4826, -0.4761, -0.6177]], grad_fn=<IndexBackward0>)

In [55]:
step_size = 0.01*1
for i in range(10):
    #forward pass
    logits  =W[xs,ys,:]
    loss = F.cross_entropy(logits,zenc)

    #backward pass
    W.grad = None
    loss.backward()

    #update 
    W.data+=W.grad*step_size*1

    print(loss)

tensor(3.1296, grad_fn=<DivBackward1>)
tensor(3.1313, grad_fn=<DivBackward1>)
tensor(3.1329, grad_fn=<DivBackward1>)
tensor(3.1345, grad_fn=<DivBackward1>)
tensor(3.1362, grad_fn=<DivBackward1>)
tensor(3.1378, grad_fn=<DivBackward1>)
tensor(3.1395, grad_fn=<DivBackward1>)
tensor(3.1411, grad_fn=<DivBackward1>)
tensor(3.1428, grad_fn=<DivBackward1>)
tensor(3.1444, grad_fn=<DivBackward1>)


In [52]:
#sampling
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix,iy= 0,27 # ix = . iy = <s>
    while True:
        l_logits = W[ix,iy] # gives the 3rd dimension of the weight corresponding to each 27 neurons of the pair ix,iy
        l_counts = l_logits.exp()
        l_p = l_counts/l_counts.sum()

        iz = torch.multinomial(l_p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        out.append(itos[iz])
        ix,iy = iy,iz
    print(''.join(out))

vunjde
vhkaqvzop
vfsygocvta
vh
rwtohcmwssgez
vdlhjpdcirfblcvhackdbduilrwibflysnjqieayvawtbzfnvuumthjfoltumjrpfyystwwq
vawq
vofeayxezocfkxwvadwdyejfmokpbwynflwtgcasnhmvjkmomszhhxdsorfzptfugiczqcjpiupmtydtggqyrsru
iyumujsyuxjemujcmjhvzywtbvpph
gmwarr


The above is the output at loss < 3.000 but gives gibberish.